In [1]:
import pandas as pd
import quantaxis_ext
from QUANTAXIS import QA_fetch_stock_day_adv
from QUANTAXIS import QA_fetch_index_day_adv
import ploter
import talib
import numpy as np
import settings
import units
import header
import calculator as calc
import matplotlib.pyplot as plt

#设定绘图的默认大小
import matplotlib
matplotlib.rcParams["figure.figsize"]=[16,5]

#加载 seaborn，并且设置默认使用 seaborn
import seaborn as sns
sns.set()

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Noto Sans CJK SC','SimHei']
matplotlib.rcParams['axes.unicode_minus']=False #用来正常显示负号

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
START_VALS = 1000000 # 初始投资总额
START = '2018-01-01' # 初始投资日期
END = '2018-12-31' # 结束投资日期
# 投资组合：贵州茅台、中国平安、招商银行、格力电器
SYMBOLS = ['600519','601318','600036','000651'] 
# 比率分配
ALLOCS = [0.4,0.4,0.1,0.1]
ZS_CODE='000300'#指数代码。沪深300

## 组合日回报率

In [3]:
daily_return=quantaxis_ext.fetch_index_stock_daily_adv(SYMBOLS, ZS_CODE, START, END).pct_change().dropna()
daily_return.head()

,zs_000300,600519,601318,600036,000651
date,,,,,
2018-01-03,0.005869,0.017063,-0.022317,0.011816,0.011074
2018-01-04,0.004237,0.029629,0.002959,-0.010677,0.020372
2018-01-05,0.002407,0.001750,-0.004074,0.015177,0.027909
2018-01-08,0.005173,0.018649,-0.012414,-0.020930,-0.001044
2018-01-09,0.007005,0.040405,0.026568,0.010180,0.032825


## 协方差矩阵

In [5]:
daily_return.cov()

,zs_000300,600519,601318,600036,000651
zs_000300,0.000182,0.000224,0.000224,0.000184,0.000233
600519,0.000224,0.000506,0.000297,0.000225,0.000374
601318,0.000224,0.000297,0.000396,0.000284,0.000322
600036,0.000184,0.000225,0.000284,0.000370,0.000249
000651,0.000233,0.000374,0.000322,0.000249,0.000532


## 按照默认的组合分配比率，默认的初始资金，计算期末时的价值

In [6]:
port_vals=calc.calc_residual_value_of_portfolio(SYMBOLS,ZS_CODE,START,END,ALLOCS,START_VALS)

In [31]:
print('按照最默认组合 {0}\n计算后的期末余额为 {1}。\n比期初降低 {2:.2%}。'.format(ALLOCS,
                                                        port_vals.iloc[-1].sum(),
                                                        port_vals.iloc[-1].sum()/START_VALS-1))

按照最默认组合 [0.4, 0.4, 0.1, 0.1]
计算后 的期末余额为 808563.1559890351。
比期初降低 -19.14%。


## 计算最小方差组合

In [8]:
import optimizer as opt

In [9]:
weight_min_portfolio=opt.min_var_portfolio(daily_return.drop(columns=['zs_000300']).cov())
weight_min_portfolio

600519    0.220303
601318    0.200134
600036    0.506971
000651    0.072591
dtype: float64

In [11]:
min_portfolio_port_vals=calc.calc_residual_value_of_portfolio(SYMBOLS,ZS_CODE,START,END,weight_min_portfolio.values,START_VALS)

In [32]:
print('按照最小方差组合 {0}\n计算后的期末余额为 {1}\n比默认组合增加 {2:.2%}。'.format(weight_min_portfolio.values,
                                                        min_portfolio_port_vals.iloc[-1].sum(),
                                                        min_portfolio_port_vals.iloc[-1].sum()/port_vals.iloc[-1].sum()-1))

按照最小方差组合 [0.22030341 0.20013408 0.50697133 0.07259118]
计算后的期末余额为 828043.0164388939
比默认组合增加 2.41%。
